In [ ]:
!pip install requests beautifulsoup4 selenium pushbullet.py schedule webdriver_manager

In [13]:
import requests
from bs4 import BeautifulSoup
import hashlib
import time
import schedule
from pushbullet import Pushbullet
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Pushbullet API key (sign up at https://www.pushbullet.com/ to get the key)
API_KEY = ''

# Initialize Pushbullet instance
pb = Pushbullet(API_KEY)

# URL of the careers portal
url = 'https://www.tu-chemnitz.de/career-service/jobboerse/index.php'  # Replace with the actual URL

In [ ]:
# Function to get the current page content for static pages
def get_static_page_content():
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Modify the selector according to the HTML structure of the page
    job_listings = soup.find_all('div', class_='job-listing')  # Example selector
    job_text = ''.join([str(job) for job in job_listings])
    return job_text

In [ ]:
# Function to get the current page content for dynamic pages (using Selenium)
def get_dynamic_page_content():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (no browser UI)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(url)
    time.sleep(5)  # Wait for JavaScript to load content (adjust as needed)

    # Modify the selector according to the HTML structure of the page
    job_listings = driver.find_elements(By.CLASS_NAME, 'job-listing')  # Example selector
    job_text = ''.join([listing.text for listing in job_listings])
    driver.quit()
    return job_text

In [ ]:
# Function to send a notification using Pushbullet
def send_notification(message):
    pb.push_note("Careers Portal Update", message)

# Function to compare and detect changes
def check_for_updates():
    # Try to get content using the static method first
    try:
        current_content = get_static_page_content()
    except Exception:
        # If static content method fails, fall back to dynamic method (Selenium)
        current_content = get_dynamic_page_content()

    # Load the previously stored content (if any)
    try:
        with open('last_content_hash.txt', 'r') as file:
            last_content_hash = file.read().strip()
    except FileNotFoundError:
        last_content_hash = ''

    # Generate the hash of the current content
    current_hash = hashlib.md5(current_content.encode()).hexdigest()

    # Compare the hash of the current content with the previous one
    if current_hash != last_content_hash:
        send_notification("The careers portal has been updated!")

        # Save the new hash
        with open('last_content_hash.txt', 'w') as file:
            file.write(current_hash)

In [15]:
# Schedule the script to run periodically (e.g., every 10 minutes)
schedule.every(10).seconds.do(check_for_updates)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 